In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import glob
import os

# Obtener la ruta de la carpeta "data"
ruta_carpeta = os.path.join(os.getcwd(), 'data')

# Obtener la lista de archivos CSV en la carpeta "data"
archivos_csv = glob.glob(os.path.join(ruta_carpeta, "*.csv"))

# Crear una lista para almacenar los DataFrames individuales
dfs = []

# Leer cada archivo CSV y almacenar su contenido en un DataFrame
for archivo in archivos_csv:
    df = pd.read_csv(archivo)
    dfs.append(df)

# Combinar los DataFrames en uno solo
df_reseñas = pd.concat(dfs, ignore_index=True)

# Mostrar el DataFrame final
df_reseñas


,User Name,User Profile,User Avatar,User Location,Rating,Rating Text,Review Title,Review Text,Helpful Votes,Photos,Published Date,Disclaimer
0,Trip26751498249,https://www.tripadvisor.com/Profile/Trip267514...,https://media-cdn.tripadvisor.com/media/photo-...,NaN,5,Jan 2024 • Family,Great Service,We used NYC LUXOR LIMO recently for our trip t...,0,NaN,"Written January 22, 2024",This review is the subjective opinion of a Tri...
1,kamal r,https://www.tripadvisor.com/Profile/727kamalr,https://media-cdn.tripadvisor.com/media/photo-...,NaN,5,Sep 2023 • Business,Excellent,Excellent service. Every driver was fully top...,0,NaN,"Written September 15, 2023",This review is the subjective opinion of a Tri...
2,Emily Smith,https://www.tripadvisor.com/Profile/12Esmith902,https://media-cdn.tripadvisor.com/media/photo-...,NaN,5,Jan 2022 • Business,Excellent ride from Boston to New York,I often travel to New York for work and I alwa...,0,NaN,"Written February 13, 2022",This review is the subjective opinion of a Tri...
3,Alex,https://www.tripadvisor.com/Profile/957manjits,https://media-cdn.tripadvisor.com/media/photo-...,NaN,5,Jan 2022 • Business,Airport service,Best car service in NYC I was provided the bes...,0,NaN,"Written February 13, 2022",This review is the subjective opinion of a Tri...
4,Tom L,https://www.tripadvisor.com/Profile/tomlY4425NE,https://media-cdn.tripadvisor.com/media/photo-...,NaN,5,Dec 2021 • Friends,LaGuardia to Manhattan,During Christmas I was traveling from LaGuardi...,0,https://media-cdn.tripadvisor.com/media/photo-...,"Written February 13, 2022",This review is the subjective opinion of a Tri...
...,...,...,...,...,...,...,...,...,...,...,...,...
220,Manuel P,https://www.tripadvisor.com/Profile/Q1748BLman...,https://media-cdn.tripadvisor.com/media/photo-...,"Vicente Lopez, Argentina",5,May 2019 • Family,Relier,Excelente todo; la atencion pre-viaje - el ase...,0,https://media-cdn.tripadvisor.com/media/photo-...,"Written May 31, 2019",This review is the subjective opinion of a Tri...
221,Valeria345,https://www.tripadvisor.com/Profile/Valeria345,https://media-cdn.tripadvisor.com/media/photo-...,"Buenos Aires, Argentina",5,May 2019 • Family,Excelente servicio!!,"Puntual, excelente servicio y atención en todo...",0,NaN,"Written May 31, 2019",This review is the subjective opinion of a Tri...
222,Lizardo P,https://www.tripadvisor.com/Profile/lizardop859,https://media-cdn.tripadvisor.com/media/photo-...,NaN,5,Feb 2019 • Solo,Muy bueno,La mejor manera de llegar a Nueva York. El tra...,0,NaN,"Written May 27, 2019",This review is the subjective opinion of a Tri...
223,Emiliano V,https://www.tripadvisor.com/Profile/emilianov753,https://media-cdn.tripadvisor.com/media/photo-...,NaN,5,May 2019 • Business,Perfecto,"Re linda energía, eficaz, muy atentos y puntua...",0,NaN,"Written May 27, 2019",This review is the subjective opinion of a Tri...


In [3]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
# Descargar recursos de NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
#La columna que tiene las reseñás, paso todos los registros a ingles

!pip install google_trans_new




In [12]:
from google_trans_new import google_translator

# Crear una instancia del traductor
translator = google_translator()

# Traducir la columna "Review_Text" de español a inglés
df_reseñas['Review_Text_english'] = df_reseñas['Review_Text'].apply(lambda x: translator.translate(x, lang_src='es', lang_tgt='en'))

# Imprimir el DataFrame con la nueva columna traducida
df_reseñas



ModuleNotFoundError: No module named 'google_trans_new'

In [21]:
!pip install requests




In [22]:
# Función para traducir el texto
def translate_text(text):
    try:
        # Hacer la solicitud a la API de Google Translate
        response = requests.get(
            'https://translate.googleapis.com/translate_a/single',
            params={
                'client': 'gtx',
                'sl': 'es',
                'tl': 'en',
                'dt': 't',
                'q': text
            }
        )
        # Extraer el texto traducido de la respuesta
        translated_text = response.json()[0][0][0]
        return translated_text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Aplicar la función a la columna Review_Text
df_reseñas['Review_Text_english'] = df_reseñas['Review_Text'].apply(translate_text)

# Ver los resultados
print(df_reseñas.head())

KeyError: 'Review_Text'

In [18]:
import mtranslate as mt
import pandas as pd

# Suponiendo que df_reseñas es tu DataFrame con la columna Review_Text que quieres traducir

# Función para traducir el texto
def translate_text(text):
    try:
        # Traducir el texto a inglés
        translated_text = mt.translate(text, 'en', 'auto')
        return translated_text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Aplicar la función a la columna Review_Text
df_reseñas['Review_Text_english'] = df_reseñas['Review_Text'].apply(translate_text)

# Ver los resultados
print(df_reseñas.head())




ModuleNotFoundError: No module named 'mtranslate'

In [ ]:
# Definir las reseñas y sus respectivas categorías
reviews = [
    ("What is the approx. TRAVEL TIME from JFK to MANHATTAN via NYC PT @ 5:35pm on a Thursday?", "preguntas sobre tiempo de viaje"),
    ("Hi Donny, can you please give me a quote for two passengers + 2 cases: pick-up in Chatham, New Jersey and transfer to Terminal 5 at JFK on Friday 5th July. Thank you so much.", "solicitud de cotización de servicio de taxi"),
    ("I’m flying into EWR with my son in June. I saw the price is 115 for a Sedan. Does the price include tolls? Does it include gratuity? And what type of cars do y’all use? Thanks so much!", "preguntas sobre detalles de precios y servicios")
]


In [ ]:


# Definir las reseñas y sus respectivas categorías
reviews = [
    ("What is the approx. TRAVEL TIME from JFK to MANHATTAN via NYC PT @ 5:35pm on a Thursday?", "preguntas sobre tiempo de viaje"),
    ("Hi Donny, can you please give me a quote for two passengers + 2 cases: pick-up in Chatham, New Jersey and transfer to Terminal 5 at JFK on Friday 5th July. Thank you so much.", "solicitud de cotización de servicio de taxi"),
    ("I’m flying into EWR with my son in June. I saw the price is 115 for a Sedan. Does the price include tolls? Does it include gratuity? And what type of cars do y’all use? Thanks so much!", "preguntas sobre detalles de precios y servicios")
]

# Preprocesamiento de datos
stop_words = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform([review[0] for review in reviews])
y = [review[1] for review in reviews]

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Evaluación del modelo
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

# Uso del modelo
new_reviews = [
    "What is the price for a ride from Manhattan to Brooklyn?",
    "Can I book a taxi for tomorrow morning?",
    "Is there any additional charge for extra luggage?"
]
X_new = vectorizer.transform(new_reviews)
predictions = classifier.predict(X_new)
print("Predicciones para nuevas reseñas:")
for review, prediction in zip(new_reviews, predictions):
    print(f"{review} -> {prediction}")
